In [1]:
import pandas as pd

---

## Loading data

In [2]:
data = pd.read_csv("../../datasets/actives_and_problematics_regarding_time_last_30_days_from-27-07-2022.csv")

In [3]:
data.shape

(9, 53)

In [4]:
list(data.columns)

['AppName',
 'ConnectionType',
 'CreatedById',
 'CreatedDate',
 'DeviceId',
 'DeviceModel',
 'DevicePlatform',
 'DeviceSessionId',
 'Duration',
 'DurationInSecs',
 'DurationInMinutes',
 'EffectivePageTime',
 'EventDate',
 'EventIdentifier',
 'OperationType',
 'OsName',
 'OsVersion',
 'PageStartTime',
 'PageUrl',
 'PreviousPageAppName',
 'PreviousPageEntityId',
 'PreviousPageEntityType',
 'PreviousPageUrl',
 'LeftQueriedEntities',
 'RecordId',
 'ReportId',
 'SdkAppType',
 'SdkAppVersion',
 'SdkVersion',
 'SessionLevel',
 'SourceIp',
 'UserId',
 'Username',
 'UserType',
 '_raw',
 '_time',
 'ColumnHeaders',
 'DashboardId',
 'DashboardName',
 'Description',
 'DisplayedFieldEntities',
 'EvaluationTime',
 'ExportFileFormat',
 'Format',
 'GroupedColumnHeaders',
 'Name',
 'NumberOfColumns',
 'OwnerId',
 'QueriedEntities',
 'RowsProcessed',
 'RowsReturned',
 'Scope',
 'Sequence']

## Loading lightning error

In [5]:
lightning_error_1 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk1.csv')

In [6]:
lightning_error_2 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk2.csv')

In [7]:
lightning_error_3 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk3.csv')

In [8]:
lightning_error_4 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk4.csv')

In [9]:
lightning_error_5 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk5.csv')

In [10]:
lightning_error_6 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk6.csv')

In [11]:
lightning_error_7 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk7.csv')

In [12]:
lightning_error_8 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk8.csv')

In [13]:
lightning_error_9 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk9.csv')

In [14]:
lightning_error_10 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk10.csv')

In [15]:
lightning_error_11 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk11.csv')

In [16]:
lightning_error_12 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk12.csv')

In [17]:
lightning_error_13 = pd.read_csv('../../data/ELF-v3/2022-07-27/2022-07-27_LightningError_chunk13.csv')

In [18]:
# frames = [lightning_error_1, lightning_error_2, lightning_error_3, lightning_error_4, lightning_error_5, 
#           lightning_error_6, lightning_error_7, lightning_error_8, lightning_error_9, lightning_error_10, 
#           lightning_error_11, lightning_error_12, lightning_error_13]

In [19]:
lightning_error_1 = pd.read_csv('../../data/ELF-v3/2022-07-22/2022-07-22_LightningError_chunk1.csv')

In [20]:
lightning_error_2 = pd.read_csv('../../data/ELF-v3/2022-07-22/2022-07-22_LightningError_chunk2.csv')

In [21]:
lightning_error_3 = pd.read_csv('../../data/ELF-v3/2022-07-22/2022-07-22_LightningError_chunk3.csv')

In [22]:
lightning_error_4 = pd.read_csv('../../data/ELF-v3/2022-07-22/2022-07-22_LightningError_chunk4.csv')

In [23]:
# frames = [lightning_error_1, lightning_error_2, lightning_error_3, lightning_error_4]

In [24]:
lightning_error_1 = pd.read_csv('../../data/ELF-v3/2022-07-11/2022-07-11_LightningError_chunk1.csv')

In [25]:
lightning_error_2 = pd.read_csv('../../data/ELF-v3/2022-07-11/2022-07-11_LightningError_chunk2.csv')

In [26]:
lightning_error_3 = pd.read_csv('../../data/ELF-v3/2022-07-11/2022-07-11_LightningError_chunk3.csv')

In [27]:
lightning_error_4 = pd.read_csv('../../data/ELF-v3/2022-07-11/2022-07-11_LightningError_chunk4.csv')

In [28]:
# frames = [lightning_error_1, lightning_error_2, lightning_error_3, lightning_error_4]

In [29]:
lightning_error_1 = pd.read_csv('../../data/ELF-v3/2022-07-05/2022-07-05_LightningError_chunk1.csv')

In [30]:
lightning_error_2 = pd.read_csv('../../data/ELF-v3/2022-07-05/2022-07-05_LightningError_chunk2.csv')

In [31]:
lightning_error_3 = pd.read_csv('../../data/ELF-v3/2022-07-05/2022-07-05_LightningError_chunk3.csv')

In [32]:
lightning_error_4 = pd.read_csv('../../data/ELF-v3/2022-07-05/2022-07-05_LightningError_chunk4.csv')

In [33]:
lightning_error_5 = pd.read_csv('../../data/ELF-v3/2022-07-05/2022-07-05_LightningError_chunk5.csv')

In [34]:
lightning_error_6 = pd.read_csv('../../data/ELF-v3/2022-07-05/2022-07-05_LightningError_chunk6.csv')

In [35]:
# frames = [lightning_error_1, lightning_error_2, lightning_error_3, lightning_error_4, lightning_error_5, 
#           lightning_error_6]

In [36]:
lightning_error_1 = pd.read_csv('../../data/ELF-v3/2022-07-04/2022-07-04_LightningError_chunk1.csv')

In [37]:
# frames = [lightning_error_1]

In [38]:
lightning_error_1 = pd.read_csv('../../data/ELF-v3/2022-06-28/2022-06-28_LightningError_chunk1.csv')

In [39]:
lightning_error_2 = pd.read_csv('../../data/ELF-v3/2022-06-28/2022-06-28_LightningError_chunk2.csv')

In [40]:
lightning_error_3 = pd.read_csv('../../data/ELF-v3/2022-06-28/2022-06-28_LightningError_chunk3.csv')

In [41]:
lightning_error_4 = pd.read_csv('../../data/ELF-v3/2022-06-28/2022-06-28_LightningError_chunk4.csv')

In [42]:
frames = [lightning_error_1, lightning_error_2, lightning_error_3, lightning_error_4]

In [43]:
lightning_error = pd.concat(frames)

In [44]:
lightning_error.shape

(231, 39)

### Getting *Report_Id* by *Page_Url*

In [45]:
lightning_error = lightning_error.dropna(subset = ['PAGE_URL'])

In [46]:
lightning_error.shape

(231, 39)

In [47]:
run_report_api_resource = "/lightning/r/"

def get_report_info_from_endpoint(page_url):
    if not(pd.isnull(page_url)) == True: 
        if page_url.startswith(run_report_api_resource):
            uri = page_url.split(run_report_api_resource)
            if len(uri) >= 2:
                report_type, report_id_derived = uri[1].split("/")[:2]
                return report_type, report_id_derived

In [48]:
lightning_error['REPORT_INFO_DERIVED'] = lightning_error.PAGE_URL.apply(get_report_info_from_endpoint)
lightning_error['REPORT_TYPE_DERIVED'] = lightning_error.REPORT_INFO_DERIVED.str[0]
lightning_error['REPORT_ID_DERIVED'] = lightning_error.REPORT_INFO_DERIVED.str[1]

In [49]:
lightning_error.shape

(231, 42)

## Merging lightning error and problematic reports

In [50]:
ltng_reports = pd.merge(left=data, right=lightning_error, left_on="ReportId", right_on="REPORT_ID_DERIVED")

In [51]:
list(ltng_reports.columns)

['AppName',
 'ConnectionType',
 'CreatedById',
 'CreatedDate',
 'DeviceId',
 'DeviceModel',
 'DevicePlatform',
 'DeviceSessionId',
 'Duration',
 'DurationInSecs',
 'DurationInMinutes',
 'EffectivePageTime',
 'EventDate',
 'EventIdentifier',
 'OperationType',
 'OsName',
 'OsVersion',
 'PageStartTime',
 'PageUrl',
 'PreviousPageAppName',
 'PreviousPageEntityId',
 'PreviousPageEntityType',
 'PreviousPageUrl',
 'LeftQueriedEntities',
 'RecordId',
 'ReportId',
 'SdkAppType',
 'SdkAppVersion',
 'SdkVersion',
 'SessionLevel',
 'SourceIp',
 'UserId',
 'Username',
 'UserType',
 '_raw',
 '_time',
 'ColumnHeaders',
 'DashboardId',
 'DashboardName',
 'Description',
 'DisplayedFieldEntities',
 'EvaluationTime',
 'ExportFileFormat',
 'Format',
 'GroupedColumnHeaders',
 'Name',
 'NumberOfColumns',
 'OwnerId',
 'QueriedEntities',
 'RowsProcessed',
 'RowsReturned',
 'Scope',
 'Sequence',
 'EVENT_TYPE',
 'TIMESTAMP',
 'REQUEST_ID',
 'ORGANIZATION_ID',
 'USER_ID',
 'CLIENT_ID',
 'SESSION_KEY',
 'LOGIN_KE

In [52]:
interest_columns = ['DurationInMinutes', 'ReportId', 'UI_EVENT_ID', 'UI_EVENT_TYPE', 'UI_EVENT_SOURCE', 'UI_EVENT_TIMESTAMP']

In [53]:
ltng_reports = ltng_reports[interest_columns]

In [54]:
ltng_reports.shape

(0, 6)